In [1]:
from src.constants import *
import os
from dataclasses import dataclass
from pathlib import Path

In [2]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_classes: int
    params_batch_size: int
    params_learning_rate: float
    params_include_top: bool
    params_weights: str

In [3]:
from src.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = params.IMAGE_SIZE,
            params_batch_size = params.BATCH_SIZE,
            params_include_top = params.INCLUDE_TOP,
            params_classes = params.CLASSES,
            params_learning_rate = params.LEARNING_RATE,
            params_weights=params.WEIGHTS

        )
        return prepare_base_model_config

In [5]:
from src import logger
import tensorflow as tf
from src.utils.common import save_model

In [8]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.base_model =  tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top)
        
        save_model(path=self.config.base_model_path, model=self.base_model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):

        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False  

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation='softmax')(flatten_in)

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate),
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                        metrics=['accuracy'])
        
        full_model.summary()
        
        return full_model
    
    def updated_base_model(self)->tf.keras.Model:
        self.full_model = self.prepare_full_model(
            model = self.base_model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        ) 
        save_model(path=self.config.updated_base_model_path, model=self.full_model)


        
        

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.updated_base_model()
except Exception as e:
    raise e    

[2023-10-16 23:03:58,447: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-16 23:03:58,450: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-16 23:03:58,451: INFO: common: created directory at: artifacts]
[2023-10-16 23:03:58,452: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-10-16 23:03:59,224: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2023-10-16 23:03:59,378: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                 